In [2]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)
library(mediation)
library(lavaan)
library(lavaanPlot)

In [3]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])
ukb_area <- ukb_df('ukb40807', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
glob2rx("area_of*_2_*") # Create a regexp to use in grep
ukbb_area=(ukb_area[c(1, grep('^area_of.*_2_',colnames(ukb_area)))])

### Exclude outliers in the imaging dataset (n=22k)
di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Mental health issues
di$schizophrenia <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$schizophrenia[di[i]=="Schizophrenia"] <- 1
di$schizophrenia[di[i]=="Depression"] <- 1
di$schizophrenia[di[i]=="Mania, hypomania, bipolar or manic-depression"] <- 1
di$schizophrenia[di[i]=="Bulimia nervosa"] <- 1
di$schizophrenia[di[i]=="Anorexia nervosa"] <- 1
di$schizophrenia[di[i]=="Attention deficit or attention deficit and hyperactivity disorder (ADD/ADHD)"] <- 1
di$schizophrenia[di[i]=="Panic attacks"] <- 1
}


# Exclude participants who have diabetes
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('diabetes_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$diabetes[di[i] =="Yes"] <- 1
}

# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i] =="High blood pressure"] <- 1
}



#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
di$excluded[di$bipolar_dis ==1] <- 1
di$excluded[di$personality_dis == 1] <- 1
di$excluded[di$schizophrenia == 1] <- 1
#di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1
#di$excluded[di$diabetes ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')
ukbb_all=merge(ukbb_all, ukbb_area, by='eid',all.x=TRUE)

nrow(ukbb_all)

ukbb_all$WMH_logscaled=log(ukbb_all$total_volume_of_white_matter_hyperintensities_from_t1_and_t2_flair_images_f25781_2_0*
                    ukbb_all$volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0)
ukbb_all$logBMI3=log(ukbb_all$body_mass_index_bmi_f21001_2_0)
ukbb_all$WHR=ukbb_all$waist_circumference_f48_2_0/ukbb_all$hip_circumference_f49_2_0

## Merge with white matter integrity variables
ukb_WM <- ukb_df('ukb35375', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
ukbb_all=merge(ukbb_all,ukb_WM,by='eid',all.x=TRUE)

#Remove outliers
variables_for_outliers=c(58:88, 120:150, 26:27, 32:38, 42:43, 48:54, 89:119, 
                         151:181, 9, 2287:2300, 2116, 5094:5096, 5120:5215,
                         grep('^area_of.*_2_',colnames(ukbb_all))) # all volume and thickness variables from FS, BMI, eTIV, area

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_all[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_all[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_all[j][ukbb_all[j]<lower]=NA
    ukbb_all[j][ukbb_all[j]>upper]=NA
}

ukbb_all=ukbb_all[complete.cases(ukbb_all[,5094:5096]),] # BMI, WMH, WHR
nrow(ukbb_all)


ukbb_all$Hypertension=0
ukbb_all$Hypertension[ukbb_all$vascular_heart_diagnoses=='1']=1
ukbb_all$Diabetes=0
ukbb_all$Diabetes[ukbb_all$diabetes=='1']=1

In [4]:
UKB_Blood <- ukb_df("ukb41240", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")

In [5]:
ukbb_blood=merge(ukbb_all,UKB_Blood, by='eid', all.x=TRUE)

In [19]:
variables_for_outliers=c(6284) 

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_blood[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_blood[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_blood[j][ukbb_blood[j]<lower]=NA
    ukbb_blood[j][ukbb_blood[j]>upper]=NA
}

In [12]:
cor.test(ukbb_blood$body_mass_index_bmi_f21001_2_0,log(ukbb_blood$creactive_protein_f30710_1_0))

In [42]:
UKBMED=data.frame(ukbb_blood$logBMI3,ukbb_blood$WMH_logscaled,log(ukbb_blood$creactive_protein_f30710_0_0),
                  ukbb_blood$sex_f31_0_0,ukbb_blood$age_when_attended_assessment_centre_f21003_2_0,
                 ukbb_blood$Hypertension,ukbb_blood$Diabetes, ukbb_blood$WHR, log(ukbb_blood$hdl_cholesterol_f30760_0_0))
colnames(UKBMED)=c('logBMI3','WMH','CRP','Sex','Age','Hypertension','Diabetes','WHR','GH')
UKBMED=na.omit(UKBMED)

In [17]:
set.seed(2014)
med.fit <- lm(scale((UKBMED$CRP)) ~
          scale(UKBMED$logBMI3)  +
          scale(UKBMED$Age) + UKBMED$Hypertension + UKBMED$Diabetes +
          UKBMED$Sex, data=UKBMED)


  
    out.fit <- lm(scale(UKBMED$WMH) ~ 
                scale((UKBMED$CRP)) +
                scale(UKBMED$logBMI3) +
                scale(UKBMED$Age) + UKBMED$Hypertension + UKBMED$Diabetes +
                UKBMED$Sex, data=UKBMED)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(UKBMED$logBMI3)',
                   mediator='scale((UKBMED$CRP))',
                   robustSE=TRUE,
                   sims=1000, data=UKBMED)
    
    med2dep <- summary(lm(UKBMED$WMH ~ 
                scale(UKBMED$CRP) +
                scale(UKBMED$Age) + UKBMED$Hypertension + UKBMED$Diabetes +
                UKBMED$Sex, data=UKBMED)) 
      

summary(med.out)
print(med2dep)


In [18]:
set.seed(2014)
med.fit <- lm(scale((UKBMED$CRP)) ~
          scale(UKBMED$WHR)  +
          scale(UKBMED$Age) + UKBMED$Hypertension + UKBMED$Diabetes +
          UKBMED$Sex, data=UKBMED)


  
    out.fit <- lm(scale(UKBMED$WMH) ~ 
                scale((UKBMED$CRP)) +
                scale(UKBMED$WHR) +
                scale(UKBMED$Age) + UKBMED$Hypertension + UKBMED$Diabetes +
                UKBMED$Sex, data=UKBMED)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(UKBMED$WHR)',
                   mediator='scale((UKBMED$CRP))',
                   robustSE=TRUE,
                   sims=1000, data=UKBMED)
    
    med2dep <- summary(lm(UKBMED$WMH ~ 
                scale(UKBMED$CRP) +
                scale(UKBMED$Age) + UKBMED$Hypertension + UKBMED$Diabetes +
                UKBMED$Sex, data=UKBMED)) 
      

summary(med.out)
print(med2dep)
